## Preprocessing

In [1]:
!pip install category_encoders
!pip install keras-tuner
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import category_encoders as ce
import pandas as pd
import tensorflow as tf
import datetime, os
import keras_tuner as kt
from keras import regularizers
import pandas as pd


In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


#Try to Optimize the Model to 75% accuracy

In [3]:
#replacing the get_dummies encoding for Income_Amt and Special_Considerations with ordinal encoding
working_df = application_df.drop(['EIN', 'NAME'], axis=1)

classifications_to_replace = working_df['CLASSIFICATION'].value_counts(dropna=False)[working_df['CLASSIFICATION'].value_counts(dropna=False)<1883]
for cls in classifications_to_replace.index:
    working_df['CLASSIFICATION'] = working_df['CLASSIFICATION'].replace(cls,"Other")

application_types_to_replace = working_df['APPLICATION_TYPE'].value_counts(dropna=False)[working_df['APPLICATION_TYPE'].value_counts(dropna=False)<528]
for app in application_types_to_replace.index:
    working_df['APPLICATION_TYPE'] = working_df['APPLICATION_TYPE'].replace(app,"Other")

dummies_df=pd.get_dummies(working_df[['ORGANIZATION', 'USE_CASE', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'STATUS']])
dummies_df['SPECIAL_CONSIDERATIONS']=working_df['SPECIAL_CONSIDERATIONS']
dummies_df['SPECIAL_CONSIDERATIONS'].replace('Y',1, inplace=True)
dummies_df['SPECIAL_CONSIDERATIONS'].replace('N',0, inplace=True)

#create an encoder
myencoder=ce.OrdinalEncoder(cols=['INCOME_AMT'], return_df=True, mapping=[{'col':'INCOME_AMT', 'mapping':{'0':0,
                                                                                                          '1-9999':1,
                                                                                                          '10000-24999':2,
                                                                                                          '25000-99999':3,
                                                                                                          '100000-499999':4,
                                                                                                          '1M-5M':5,
                                                                                                          '5M-10M':6,
                                                                                                          '10M-50M':7,
                                                                                                          '50M+':8}}])
#fitting and Transforming income data
dummies_df['INCOME_AMT']=myencoder.fit_transform(working_df['INCOME_AMT'])
#checking the result
dummies_df['INCOME_AMT'].value_counts()


INCOME_AMT
0    24388
3     3747
4     3374
5      955
1      728
2      543
7      240
6      185
8      139
Name: count, dtype: int64

In [4]:
dummies_df['SPECIAL_CONSIDERATIONS'].value_counts()

SPECIAL_CONSIDERATIONS
0    34272
1       27
Name: count, dtype: int64

In [5]:
#recreating train and test sets and rescaling
X=dummies_df.copy()
X['ASK_AMT']=application_df['ASK_AMT']
y=working_df['IS_SUCCESSFUL']
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
new_scaler = StandardScaler()

# Fit the StandardScaler
new_X_scaler = new_scaler.fit(X_train)

# Scale the data
X_train_scaled = new_X_scaler.transform(X_train)
X_test_scaled = new_X_scaler.transform(X_test)

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ORGANIZATION_Association      34299 non-null  bool 
 2   ORGANIZATION_Co-operative     34299 non-null  bool 
 3   ORGANIZATION_Corporation      34299 non-null  bool 
 4   ORGANIZATION_Trust            34299 non-null  bool 
 5   USE_CASE_CommunityServ        34299 non-null  bool 
 6   USE_CASE_Heathcare            34299 non-null  bool 
 7   USE_CASE_Other                34299 non-null  bool 
 8   USE_CASE_Preservation         34299 non-null  bool 
 9   USE_CASE_ProductDev           34299 non-null  bool 
 10  APPLICATION_TYPE_Other        34299 non-null  bool 
 11  APPLICATION_TYPE_T10          34299 non-null  bool 
 12  APPLICATION_TYPE_T19          34299 non-null  bool 
 13  APPLICATION_TYPE_T3           3

In [7]:
#creat a new model with kernel regularization applied
new_nn = tf.keras.models.Sequential()

# First hidden layer
new_nn.add(tf.keras.layers.Dense(units=26, activation='relu', input_dim=34, kernel_regularizer=regularizers.L1(.001)))

# Second hidden layer
new_nn.add(tf.keras.layers.Dense(units=31, activation='relu', kernel_regularizer=regularizers.L1(.001)))

#adding Third hidden layer
new_nn.add(tf.keras.layers.Dense(units=16, activation='relu', kernel_regularizer=regularizers.L1(.001)))

# #adding forth hidden layer
new_nn.add(tf.keras.layers.Dense(units=11, activation='relu', kernel_regularizer=regularizers.L1(.001)))

# Output layer
new_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
new_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 26)                910       
                                                                 
 dense_1 (Dense)             (None, 31)                837       
                                                                 
 dense_2 (Dense)             (None, 16)                512       
                                                                 
 dense_3 (Dense)             (None, 11)                187       
                                                                 
 dense_4 (Dense)             (None, 1)                 12        
                                                                 
Total params: 2458 (9.60 KB)
Trainable params: 2458 (9.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
new_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logdir = os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d=%H%M%S"))
new_weight_callback = tf.keras.callbacks.ModelCheckpoint(logdir, epoch=5, save_weights_only=True, monitor='val_accuracy', save_best_only = False)

In [9]:
new_nn_fit=new_nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[new_weight_callback])

Epoch 1/50
804/804 [==============================] - 9s 6ms/step - loss: 0.7974 - accuracy: 0.7091
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6514 - accuracy: 0.7243
Epoch 3/50
804/804 [==============================] - 6s 8ms/step - loss: 0.6164 - accuracy: 0.7267
Epoch 4/50
804/804 [==============================] - 5s 6ms/step - loss: 0.6007 - accuracy: 0.7259
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5929 - accuracy: 0.7275
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5879 - accuracy: 0.7266
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5851 - accuracy: 0.7291
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5833 - accuracy: 0.7285
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5822 - accuracy: 0.7285
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5806 - accuracy: 0.7276

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = new_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5711 - accuracy: 0.7313 - 579ms/epoch - 2ms/step
Loss: 0.571112871170044, Accuracy: 0.7313119769096375


In [11]:
# Export our model to HDF5 file
new_nn.save('AlphabetSoupCharity_Optimization.h5', overwrite=True, save_format="h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Using Keras Tuner to optimize the model

In [ ]:
##using a keras tuner to optimize the model.
def create_my_model(hp):
  nn_my_model = tf.keras.models.Sequential()
  # Allow kerastuner to decide which activation function to use in hidden layers
  activation = hp.Choice('activation',['relu','leaky_relu','gelu'])
  # Allow kerastuner to decide number of neurons in first layer
  nn_my_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
      min_value=1,
      max_value=35,
      step=5), activation=activation, kernel_regularizer=regularizers.L1(.001), input_dim=34))
  # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
  for i in range(hp.Int('num_layers', 1, 4)):
      nn_my_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
          min_value=1,
          max_value=35,
          step=5),
          activation=activation,
          kernel_regularizer=regularizers.L1(.001)))

  nn_my_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

  # Compile the model
  nn_my_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

  return nn_my_model

In [ ]:
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_my_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2)

In [ ]:
tuner.search(X_train_scaled,y_train,epochs=30,validation_data=(X_test_scaled,y_test))

In [ ]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

In [ ]:
first_hyper = tuner.get_best_hyperparameters(1)[0]
first_hyper.values

In [ ]:
first_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = first_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Get second best model hyperparameters
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

In [ ]:
# Compare the performance to the second-best model
second_model = tuner.get_best_models(2)[1]
model_loss, model_accuracy = second_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs